# CSP Planning for the Course Planning Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/S24-CS143AI/blob/main/CSP_course_planning_problem.ipynb)

Every student needs to plan ahead to make sure that they will be able to fulfill all of their graduation requirements. For our next project, we'll attempt to formulate this *custom 4-year plan* problem as a Constrain Satisfaction Problem. 

To start, we will discuss challenges presented by this problem and ideas of how they can be overcome. This notebook merely shows you how to access the data that you'll use when you do the project.

To start, install the `requests` library to access data via a URL. If you will be using this notebook to start your CSP formulation, you will also need to install `python-constraint`.

In [ ]:
import sys

!{sys.executable} -m pip install python-constraint
!{sys.executable} -m pip install requests

## Data hosted on GitHub

The data we'll use is hosted on GitHub at the following URLs. Take a moment and copy-and-paste these into your browser to see what the files look like.

If you prefer, you could just download these files and access them locally with the normal Python file IO tools, like `open("course_rotation.json","r")`. The URLs are meant to make it easy for us to all share the same files and make changes if needed.

In [1]:
import requests
import json

course_rotation_url = 'https://raw.githubusercontent.com/ericmanley/S24-CS143AI/main/data/course_rotation.json'
cs_requirements_url = 'https://raw.githubusercontent.com/ericmanley/S24-CS143AI/main/data/cs_major_requirements.json'
prerequisites_url = 'https://raw.githubusercontent.com/ericmanley/S24-CS143AI/main/data/prerequisites_simplified.json'

## Course Rotation Data

The course rotation data is a list of dictionaries. Each dictionary has two keys:
* `'course'` - this is the course/number for a real course at Drake
* `'terms_offered'` - this a list of upcoming terms when the course is likely to be offered
    - 202505 means Summer of 2024 (academic year 2025 runs Summer 2024 through Spring 2025)
    - 202510 means Fall 2024
    - 202515 means January 2025
    - 202520 means Spring 2025

In [6]:
course_rotation_data = requests.get(course_rotation_url).json()
course_rotation_data

[{'course': 'CS 065',
  'terms_offered': [202510,
   202520,
   202610,
   202620,
   202710,
   202720,
   202810,
   202820]},
 {'course': 'CS 066',
  'terms_offered': [202510,
   202520,
   202610,
   202620,
   202710,
   202720,
   202810,
   202820]},
 {'course': 'CS 067',
  'terms_offered': [202510,
   202520,
   202610,
   202620,
   202710,
   202720,
   202810,
   202820]},
 {'course': 'CS 083',
  'terms_offered': [202510,
   202515,
   202520,
   202610,
   202615,
   202620,
   202710,
   202715,
   202720,
   202810,
   202815,
   202820]},
 {'course': 'CS 120', 'terms_offered': [202520, 202620, 202720, 202820]},
 {'course': 'CS 130', 'terms_offered': [202510, 202610, 202710, 202810]},
 {'course': 'CS 135', 'terms_offered': [202620, 202820]},
 {'course': 'CS 137', 'terms_offered': [202510, 202610, 202710, 202810]},
 {'course': 'CS 139', 'terms_offered': [202620, 202820]},
 {'course': 'CS 143', 'terms_offered': [202520, 202620, 202720, 202820]},
 {'course': 'CS 150', 'terms

## CS Major Requirements

This data is also a list of dictionaries. Each dictionary has three keys:
* `'requirement_name'` - this is just a way to identify each requirement. In cases where the requirement is a specific course, it might have the same name as a course (e.g., `'CS 065'`). Sometimes, there are options, like the CS requirement that you take either Math 54 or Math 100 - I've given that the name `'Math Proof'`.
* `'courses'` - this is the list of courses that can fulfill this requirement (e.g., `['MATH 054', 'MATH 101']` for the `'Math Proof'` requirement).
* `'num_required'` - this is the number of courses from the `'courses'` list that students are required to take. Most requirements only need 1 course, but some are lists of electives like `'Upper Division Component'` in which you need to take 5 from the list.

In [8]:
cs_requirements_data = requests.get(cs_requirements_url).json()
cs_requirements_data

[{'requirement_name': 'CS 065', 'num_required': 1, 'courses': ['CS 065']},
 {'requirement_name': 'CS 066', 'num_required': 1, 'courses': ['CS 066']},
 {'requirement_name': 'CS 067', 'num_required': 1, 'courses': ['CS 067']},
 {'requirement_name': 'CS 083', 'num_required': 1, 'courses': ['CS 083']},
 {'requirement_name': 'CS 130', 'num_required': 1, 'courses': ['CS 130']},
 {'requirement_name': 'CS 137', 'num_required': 1, 'courses': ['CS 137']},
 {'requirement_name': 'CS 188', 'num_required': 1, 'courses': ['CS 188']},
 {'requirement_name': 'CS 191', 'num_required': 1, 'courses': ['CS 191']},
 {'requirement_name': 'MATH 050', 'num_required': 1, 'courses': ['MATH 050']},
 {'requirement_name': 'Math Proof',
  'num_required': 1,
  'courses': ['MATH 054', 'MATH 101']},
 {'requirement_name': 'Upper Division Component',
  'num_required': 5,
  'courses': ['CS 114',
   'CS 116',
   'CS 128',
   'CS 135',
   'CS 139',
   'CS 143',
   'CS 146',
   'CS 147',
   'CS 150',
   'CS 160',
   'CS 161',

## Prerequisite Data

This data is also a list of dictionaries. Each dictionary contains two keys:
* `'course'` - this is the course/number for a real course at Drake
* `'prerequisites'` - this is the list of courses that must be completed before a student can take the given course. This represents a simplified list that is relevant to most CS majors at Drake. However, the requirements are actually a little more complicated - for example, CS 167's requirements are (CS 065 AND (CS 066 OR STAT 40)). If you want to take a crack at data with more complicated prereqs, check out this URL instead: https://raw.githubusercontent.com/ericmanley/S24-CS143AI/main/data/prerequisites.json

In [9]:
prerequisites_data = requests.get(prerequisites_url).json()
prerequisites_data

[{'course': 'CS 066', 'prerequisites': ['CS 065']},
 {'course': 'CS 067', 'prerequisites': ['CS 066']},
 {'course': 'CS 116', 'prerequisites': ['CS 065']},
 {'course': 'CS 120', 'prerequisites': ['CS 065']},
 {'course': 'CS 128', 'prerequisites': ['CS 065', 'MATH 050']},
 {'course': 'CS 130', 'prerequisites': ['CS 066']},
 {'course': 'CS 135', 'prerequisites': ['CS 067']},
 {'course': 'CS 137', 'prerequisites': ['CS 067', 'MATH 050', 'MATH 054']},
 {'course': 'CS 139', 'prerequisites': ['CS 065', 'MATH 050', 'MATH 054']},
 {'course': 'CS 143', 'prerequisites': ['CS 066']},
 {'course': 'CS 147', 'prerequisites': ['CS 067']},
 {'course': 'CS 150', 'prerequisites': ['CS 065', 'MATH 101']},
 {'course': 'CS 160', 'prerequisites': ['CS 130']},
 {'course': 'CS 161', 'prerequisites': ['CS 130', 'CS 135']},
 {'course': 'CS 165', 'prerequisites': ['CS 065', 'MATH 080', 'MATH 100']},
 {'course': 'CS 167', 'prerequisites': ['CS 065', 'CS 066']},
 {'course': 'CS 172', 'prerequisites': ['CS 130']},


## Group Discussions

Plan for solving this problem by discussing the following:
* What should the CSP **variables** be for this problem?
* What should the **domains** be for those variables?
* Describe, in english, some constraints that you think will be necessary to include. Can you write those in terms of the variables that you decided on?

## How big do we want to make this project?

The data I've given you is primarily for *CS Majors* at Drake. If you would prefer to do a different major, that would be awesome! Just create a new file like `cs_major_requirements.json` for the other major and that major's courses to a copy of `course_rotation.json` and `prerequisites.json`. **Send these files to me so I can share them on the GitHub repository.**

There are many extensions that would be great to handle
* multiple majors/minors for one student
* AOIs
* parse a student's degree evaluation report and output a customized plan for fulfilling the rest of their requirements